# Space X Falcon 9 First Stage Landing Prediction
## Part 1A: Data Collection (Web Scraping)

 In this lab, we will be performing web scraping to collect Falcon 9 historical launch records from Wikipedia as well as parsing said data into a dataframe.</br>
 Specifically, we will be scraping the following page: https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

----

Importing our libraries:

In [1]:
import sys
import requests
import re
import unicodedata
import pandas as pd
from bs4 import BeautifulSoup

Defining our helper functions to process HTML table.

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


### 1. Requesting the Falcon9 Launch Wiki page from its URL

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
response.status_code

200

Creating our BeautifulSoup object from our response.

In [4]:
soup = BeautifulSoup(response.content)
# printing title to verify object was created correctly
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### 2. Extracting all column/variable names from HTML table header

In [5]:
html_tables = soup.find_all('table')

In [6]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

We can find the column names embedded in the `<th>` elements.</br>
In order to retrieve these, we will iterate through these elements and apply the helper function `extract_column_from_header()` one by one.

In [7]:
column_names = []

th_elements = first_launch_table.find_all('th')
for el in th_elements:
    name = extract_column_from_header(el)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [8]:
# verifying extracted columns
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

### 3. Creating dataframe by parsing launch HTML tables

First let's initialize an empty dictionary with each value being an empty list, which will eventually be converted into our dataframe.

In [9]:
launch_dict = dict.fromkeys(column_names)
# removing irrelevant column
del launch_dict['Date and time ( )']
# init dictionary w/ empty lists as values
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# adding some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Now we need to populate the `launch_dict` with records extracted from table rows. 

We need to consider, however, that usually HTML tables in Wikipedia often contain unexpected annotations and other types or noise.

In [10]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            datatimelist=date_time(row[0])
            # Flight No.
            # appending the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            
            # Date
            # appending date into launch_dict with key 'Date'
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time
            # appending time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
      
            # Booster version
            # appending bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # appending launch_site into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # appending payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # appending payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # appending the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # appending customer into launch_dict with key `Customer`
            if (row[6].a is not None):
                customer=row[6].a.string
            else:
                customer=row[6].string.strip()
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            # appending launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # appending booster_landing into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8]) 
            launch_dict['Booster landing'].append(booster_landing)

Now that we have parsed our correspondind table values into `launch_dict`, we can create a dataframe out of it.

In [11]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

Lastly, we will saved our progress for the next steps by exporting our new data frame into a CSV file.

In [12]:
df.to_csv('spacex_web_scraped.csv', index=False)